In [108]:
import pandas as pd
import glob
import re
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import hex_to_rgb
from nhl_themes import teamColours
pd.set_option('mode.chained_assignment', None)

In [109]:
data = pd.read_csv("EH_std_gl_stats_all_regular_adj_2024-02-08.csv")
data = data.loc[data["xGA"] > data["xGA"].quantile(0.25)]
data["Player"] = data["Player"].str.split(n=1).str[1]
data = data.set_index(["Player"])

In [110]:
skinner = pd.read_csv('goalie-csv/EH_game_log_stuart_skinner_on-ice_all_regular_2024-01-30.csv')
skinnerStart = skinner.head(13)
skinnerEnd = skinner.tail(21)
skinnerEnd.drop(index=34, inplace=True)
skinnerEnd.tail()

,Player,Date,Game_ID,Season,Team,Opponent,Is_Home,Position,TOI,TOI%,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAx
29,Stuart Skinner,2024-01-16,2023020688,20232024,EDM,TOR,1,G,59.70,99.50,2.13,29.79,45.15,4.10,92.84,95.28,90.93,4.35,1.97
30,Stuart Skinner,2024-01-18,2023020698,20232024,EDM,SEA,1,G,60.00,100.00,2.13,27.46,39.19,2.79,92.24,94.56,92.88,1.68,0.66
31,Stuart Skinner,2024-01-20,2023020714,20232024,EDM,CGY,0,G,59.85,99.75,0.94,25.35,32.65,2.14,96.30,97.13,93.44,3.69,1.20
32,Stuart Skinner,2024-01-23,2023020736,20232024,EDM,CBJ,1,G,60.00,100.00,1.01,27.12,40.30,3.62,96.27,97.49,91.03,6.46,2.60
33,Stuart Skinner,2024-01-27,2023020761,20232024,EDM,NSH,1,G,60.00,100.00,1.01,27.68,46.64,4.24,96.34,97.83,90.91,6.92,3.23


In [111]:
data.loc[f'Skinner Before Nov 23'] = ['23-24', 'EDM', 'G', len(skinnerStart.index), skinnerStart['TOI'].sum(), skinnerStart['GA'].sum(), 
                            skinnerStart['SA'].sum(), skinnerStart['FA'].sum(), skinnerStart['xGA'].sum(), skinnerStart['Sv%'].mean(),
                            skinnerStart['FSv%'].mean(), skinnerStart['xFSv%'].mean(), 0, 0, skinnerStart['GSAx'].sum()]

data.loc[f'Skinner After Nov 23'] = ['23-24', 'EDM', 'G', len(skinnerEnd.index), skinnerEnd['TOI'].sum(), skinnerEnd['GA'].sum(), 
                            skinnerEnd['SA'].sum(), skinnerEnd['FA'].sum(), skinnerEnd['xGA'].sum(), skinnerEnd['Sv%'].mean(),
                            skinnerEnd['FSv%'].mean(), skinnerEnd['xFSv%'].mean(), 0, 0, skinnerEnd['GSAx'].sum()]
data.tail()

,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx
Player,,,,,,,,,,,,,,,
Luukkonen,23-24,BUF,G,27,1509.63,64.82,731.20,1081.32,78.13,91.140000,94.010000,92.770000,1.23,5.50,13.31
Husso,23-24,DET,G,18,1004.12,58.05,544.41,818.41,60.62,89.340000,92.910000,92.590000,0.31,-5.70,2.57
Vanecek,23-24,N.J,G,30,1669.93,90.66,806.34,1138.77,78.83,88.760000,92.040000,93.080000,-1.04,-13.12,-11.83
Skinner Before Nov 23,23-24,EDM,G,13,705.11,43.06,326.84,470.65,35.04,85.146923,89.609231,92.692308,0.00,0.00,-8.03
Skinner After Nov 23,23-24,EDM,G,20,1202.08,37.43,534.56,759.54,55.57,92.700500,94.786500,92.705000,0.00,0.00,18.11


In [112]:
data["GSAx/60"] = round((60 * data["GSAx"]) / data["TOI"], 2)
data.head()

,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx,GSAx/60
Player,,,,,,,,,,,,,,,,
Hill,23-24,VGK,G,18,1021.13,32.17,505.96,766.12,52.10,93.64,95.80,93.20,2.60,16.49,19.93,1.17
Schmid,23-24,N.J,G,15,755.53,41.62,386.96,536.85,38.61,89.24,92.25,92.81,-0.56,-4.40,-3.01,-0.24
Lyon,23-24,DET,G,21,1192.97,50.03,638.11,902.90,60.62,92.16,94.46,93.29,1.17,11.33,10.59,0.53
Nedeljkovic,23-24,PIT,G,16,898.98,38.96,469.03,685.64,43.44,91.69,94.32,93.66,0.65,6.15,4.48,0.30
Georgiev,23-24,COL,G,42,2456.87,117.61,1152.36,1628.15,119.14,89.79,92.78,92.68,0.09,-6.79,1.53,0.04


In [113]:
data["GA/100 xGA"] = round((100 * data["GA"] / data["xGA"]))
data.head()

,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx,GSAx/60,GA/100 xGA
Player,,,,,,,,,,,,,,,,,
Hill,23-24,VGK,G,18,1021.13,32.17,505.96,766.12,52.10,93.64,95.80,93.20,2.60,16.49,19.93,1.17,62.0
Schmid,23-24,N.J,G,15,755.53,41.62,386.96,536.85,38.61,89.24,92.25,92.81,-0.56,-4.40,-3.01,-0.24,108.0
Lyon,23-24,DET,G,21,1192.97,50.03,638.11,902.90,60.62,92.16,94.46,93.29,1.17,11.33,10.59,0.53,83.0
Nedeljkovic,23-24,PIT,G,16,898.98,38.96,469.03,685.64,43.44,91.69,94.32,93.66,0.65,6.15,4.48,0.30,90.0
Georgiev,23-24,COL,G,42,2456.87,117.61,1152.36,1628.15,119.14,89.79,92.78,92.68,0.09,-6.79,1.53,0.04,99.0


In [114]:
fig = px.scatter(data, x='xGA', y='GA/100 xGA')
for row in data.index:
    fig.add_annotation(x=data.loc[row, 'xGA'], y=data.loc[row, 'GA/100 xGA'], 
            text=row, showarrow=False, name='Team',
            font=dict(size=11, color=teamColours[data.loc[row, 'Team']]['text']),
            bgcolor=teamColours[data.loc[row, 'Team']]['colour'], opacity=0.8, 
            borderpad=2.5)
fig.update_layout(showlegend=False, width=750, height=750, 
                  xaxis_title='xGA Faced',
                  yaxis_title='Goals Against per 100 xGA')
fig.update_xaxes(range=[(data["xGA"].min() - 5), (data["xGA"].max() + 5)])
fig.update_yaxes(autorange="reversed")
fig.show()